# #5. Black-Litterman model

In [1]:
import matplotlib.pylab as plt
import numpy as np
from numpy.linalg import inv
import pandas as pd
from pandas_datareader import data as web
from scipy.optimize import minimize

In [2]:
# 최적의 비중 계산을 위한 목적함수
# obj. funciton for optimizing weights
def solveWeights(R, C, rf):
    def obj(W, R, C, rf):
        mean = sum (R * W)
        var = np.dot(np.dot(W, C), W)
        
        # Sharpe's ratio as utility function: want to maximize
        util = (mean -rf) / np.sqrt(var)
        return 1 / util
    
    # number of assets
    n = len(R)
    
    # initial weights
    W = np.ones([n]) / n
    
    # doesn't allow short-selling nor borrowing
    # 비중 범위는 0-100% 사이: 공매도나 차입조건이 없음
    bnds = [(0., 1.) for i in range(n)]
    
    # constraints: sum of weights = 1
    cons = ({'type': 'eq', 'fun': lambda W: sum(W) - 1.})

    res = minimize(obj, W, (R, C, rf), method='SLSQP', constraints=cons, bounds=bnds)

    if not res.success:
        raise BaseException(res.message)
    
    return res.x

In [ ]:
# 무위험 수익률, 수익률, Cov로 효율적 투자선 계산
# calculating Efficient Frontier via risk-less rate, return rate and the Covariance
def solveFrontier(R, C, rf):

    # 최적 비중 계산을 위한 목적함수
    def obj(W, R, C, r):
        mean = sum(R * W)
        var = np.dot(np.dot(W, C), W)

        # 
        penalty = 100 * abs(mean - r)
        return var + penalty
    
    frontier_mean, frontier_var = [ ], [ ]
    n = len(R)

    for r in np.linspace(min(R), max(R), num=20):
        W = np.ones([n]) / n